<a href="https://colab.research.google.com/github/taemmini/2023_Fall_HUFS_ML_Project_7/blob/main/CAPTCHA_Project_CRNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import urllib.request
from PIL import Image, ImageFilter

def convert_and_blur(input_image_path, output_image_path):
    img = Image.open(input_image_path)
    new_img = Image.new("RGB", img.size, (255, 255, 255))
    new_img.paste(img, (0, 0), img)

    blurred_image = new_img.filter(ImageFilter.GaussianBlur(radius=2))
    blurred_image.save(output_image_path, "PNG")

input_image_path = r"C:\Users\yongh\Desktop\MLTeamProject\resource\Macro_Test\CaptchaImage.png"
output_image_path = r"C:\Users\yongh\Desktop\MLTeamProject\resource\Macro_Test\CaptchaImage_converted.png"

driver = webdriver.Chrome()
driver.get('https://ticket.melon.com/main/index.htm')

In [45]:
# 수동 로그인 후 자동화

# 예매 버튼을 클릭하고, 팝업 창으로 focus를 전환
driver.find_element_by_id('ticketReservation_Btn').click()
time.sleep(0.3)
print(driver.window_handles)
handle = driver.window_handles[1]
driver.switch_to.window(handle)

driver.set_window_position(0, 0)

# 팝업 창에서 CAPTCHA 이미지를 찾아 지정된 경로로 다운로드
image = driver.find_element_by_id('captchaImg')
url = image.get_attribute('src')
urllib.request.urlretrieve(url, r"C:\Users\yongh\Desktop\MLTeamProject\resource\Macro_Test\CaptchaImage.png")

# 다운받은 이미지를 RGBA->RGB로 전환하고 가우시안 블러처리
time.sleep(0.1)
convert_and_blur(input_image_path, output_image_path)

# 
time.sleep(0.1)
captcha_answer = get_prediction(prediction_model, output_image_path)
captcha_input = driver.find_element_by_class_name('inputType')
captcha_input.send_keys(captcha_answer)
summit_button = driver.find_element_by_id('btnComplete').click()

# back to main
main_window = driver.window_handles[0]
driver.switch_to.window(main_window)

['31C0AF49AE92ED12BD2C6CBE4FFC4C17', 'F9F65D69558FF4ADD701C5042AEA27FD']
1/1 [==============================] - 0s 32ms/step


In [31]:
# back to main

main_window = driver.window_handles[0]
driver.switch_to.window(main_window)

In [60]:
# go to pop-up window

handle = driver.window_handles[1]
driver.switch_to.window(handle)

In [ ]:
# 이하 학습 모델 로딩 관련

In [1]:
import os
from glob import glob

import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf 
from tensorflow import keras
from keras import layers

import cv2
from sklearn.model_selection import train_test_split

In [2]:
def load_image_with_label(folder_path : str, extension : str):
    images = glob(r".\resource\%s\*.%s"%(folder_path, extension))
    labels = [img.split(os.path.sep)[3].split('.'+extension)[0].split('_')[0] for img in images]
    return (images, labels)

In [3]:
images, labels = load_image_with_label('MelonCaptcha_blurred','png')
unique_char = sorted(list(set(''.join(labels))))
print(unique_char)
print(len(unique_char))

char_to_num = layers.experimental.preprocessing.StringLookup(
    vocabulary = list(unique_char), num_oov_indices = 0, mask_token = None
)
num_to_char = layers.experimental.preprocessing.StringLookup(
    vocabulary = char_to_num.get_vocabulary(), num_oov_indices = 0, mask_token = None, invert = True
)

['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z']
26



C:\Users\yongh\anaconda3\Lib\site-packages\numpy\core\numeric.py:2468: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return bool(asarray(a1 == a2).all())


In [4]:
img_width = 280
img_height = 80

def encode_image_only(path):
  img = tf.io.read_file(path)
  img = tf.io.decode_png(img, channels=1)
  img = tf.image.convert_image_dtype(img, tf.float32)
  img = tf.image.resize(img, [img_height, img_width])
  img = tf.transpose(img, perm=[1,0,2])
    
  return img

In [5]:
class CTCLayer(layers.Layer):
    def __init__(self, name=None):
        super().__init__(name=name)
        self.loss_fn = keras.backend.ctc_batch_cost

    def call(self, y_true, y_pred):
        # Compute the training-time loss value and add it
        # to the layer using `self.add_loss()`.
        batch_len = tf.cast(tf.shape(y_true)[0], dtype='int64')
        input_length = tf.cast(tf.shape(y_pred)[1], dtype='int64')
        label_length = tf.cast(tf.shape(y_true)[1], dtype='int64')

        input_length = input_length * tf.ones(shape=(batch_len, 1), dtype='int64')
        label_length = label_length * tf.ones(shape=(batch_len, 1), dtype='int64')

        loss = self.loss_fn(y_true, y_pred, input_length, label_length)
        self.add_loss(loss)

        # At test time, just return the computed predictions
        return y_pred

def build_model():
    input_img = layers.Input(
        shape=(img_width, img_height, 1), name='image', dtype='float32'
    )
    labels = layers.Input(name='label', shape=(None,), dtype='float32')

    # Convolution, Maxpooling을 각각 두번하여 차원을 축소함

    x = layers.Conv2D(
        32,
        (3, 3),
        activation='relu',
        kernel_initializer='he_normal',
        padding='same',
        name='Conv1',
    )(input_img)
    x = layers.MaxPooling2D((2, 2), name='pool1')(x)

    x = layers.Conv2D(
        64,
        (3, 3),
        activation='relu',
        kernel_initializer='he_normal',
        padding='same',
        name='Conv2',
    )(x)
    x = layers.MaxPooling2D((2, 2), name='pool2')(x)

    new_shape = ((img_width // 4), (img_height // 4) * 64)
    x = layers.Reshape(target_shape=new_shape, name='reshape')(x)
    x = layers.Dense(64, activation='relu', name='dense1')(x)
    x = layers.Dropout(0.2)(x)

    # RNN을 두 차례 사용
    x = layers.Bidirectional(layers.LSTM(128, return_sequences=True, dropout=0.25))(x)
    x = layers.Bidirectional(layers.LSTM(64, return_sequences=True, dropout=0.25))(x)

    # 출력층 구성
    x = layers.Dense(
        len(char_to_num.get_vocabulary()) + 1, activation='softmax', name='dense2'
    )(x)

    # CTC 손실함수 적용
    output = CTCLayer(name='ctc_loss')(labels, x)

    # 모델 구성
    model = keras.models.Model(
        inputs=[input_img, labels], outputs=output, name='ocr_model_v1'
    )
    # 케라스에서 제공하는 최적화
    opt = keras.optimizers.Adam()

    model.compile(optimizer=opt)
    return model

In [6]:
model = build_model()
model.load_weights('.\GaussianBlurBest_in30Epoch')

In [7]:
prediction_model = keras.models.Model(
  model.get_layer(name='image').input, model.get_layer(name='dense2').output
)

In [8]:
def decode_batch_predictions(pred):
    input_len = np.ones(pred.shape[0]) * pred.shape[1]
    results = keras.backend.ctc_decode(pred, input_length=input_len, greedy=True)[0][0][:, :max_length]
    output_text = []
    # encode된 글자를 복원하여 리스트에 저장
    for res in results:
        res = tf.strings.reduce_join(num_to_char(res)).numpy().decode('utf-8')
        output_text.append(res)
    return output_text

In [9]:
def get_prediction(model, path : str):
    image_tensor = encode_image_only(path)
    image_tensor = tf.expand_dims(image_tensor,0)
    pred = model.predict(image_tensor)
    pred_text = decode_batch_predictions(pred)
    
    return pred_text[0]

In [10]:
max_length = 6

In [13]:
print(get_prediction(prediction_model, output_image_path))

1/1 [==============================] - 1s 820ms/step

DNEOAN
